# Setups, Installations and Imports

In [1]:
%%capture 
!pip install tensorflow
!pip install tensorflow-addons
!pip install bayesian-optimization

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import resnet50

import tensorflow_addons as tfa
import sys

In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except:
    IN_COLAB = False

In [4]:
import os
os.environ["TF_DETERMINISTIC_OPS"] = "1"
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import io
import itertools
from functools import partial
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, recall_score
from tqdm.notebook import tqdm_notebook
from ipywidgets import IntProgress

from sklearn.model_selection import KFold, StratifiedKFold

from bayes_opt import BayesianOptimization

from sklearn.preprocessing import LabelBinarizer


# Download and Prepare Dataset

In [5]:
# Insert the directoryimport sys
if IN_COLAB:
    '''this is the exact path of the mounted direttory. in order to set it, right click on colab notebooks folder and copy path, and insert below '''
    COLAB_NOTEBOOKES_PATH = "/content/drive/MyDrive/Colab Notebooks"
    sys.path.insert(0,COLAB_NOTEBOOKES_PATH)  

import Ensembles_prepare_datasets


The syntax of the command is incorrect.
'touch' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'cat' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


#### Dataloader

In [6]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

def preprocess_image(image, label):
  img = tf.cast(image, tf.float32)
  img = img/255.

  return img, label


def get_loaders(x_train, y_train, x_test, y_test):
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    trainloader = (
        trainloader
        .shuffle(1024)
        .map(preprocess_image, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    testloader = (
        testloader
        .map(preprocess_image, num_parallel_calls=AUTO)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )
    return trainloader, testloader

In [7]:
ROOT_PATH = './'
## SmallCNN models with different initialization
models_dir = 'IndependentSolutions'
MODEL_PATH = ROOT_PATH+'IndependentSolutions/'

if models_dir not in os.listdir(ROOT_PATH):
    os.mkdir(models_dir)

# Model

In [8]:
def Model1(num_labels, img_shape, color = 3):
  inputs = keras.layers.Input(shape=(img_shape, img_shape, color))

  x = keras.layers.Conv2D(16, (3,3), padding='same')(inputs)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.Conv2D(32,(3,3), padding='same')(x)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.Conv2D(32,(3,3), padding='same')(x)
  x = keras.activations.relu(x)
  x = keras.layers.MaxPooling2D(2, strides=2)(x)

  x = keras.layers.GlobalAveragePooling2D()(x)
  x = keras.layers.Dense(32, activation='relu')(x)
  x = keras.layers.Dropout(0.1)(x)
  
  outputs = keras.layers.Dense(num_labels, activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

def Model():
    if color:
        return Model1(num_labels, img_shape,color)
    else:
        return Model1(num_labels, img_shape)

# Init hyper parameters

In [9]:

learning_rate_range = [0.001,0.1]
epsilon_range = [1e-08,1e-05]


# Callbacks

#### LR Scheduler

In [10]:

def lr_schedule(epoch,lr):
    if (epoch >= 0) & (epoch < 9):
        return lr
    elif (epoch >= 9) & (epoch < 19):
        return lr/2
    elif (epoch >= 19) & (epoch < 29):
        return lr/4
    else:
        return lr/8

#lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lr_schedule(epoch), verbose=True)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

# Independent Solutions

In [11]:
# Saving 5 independent solutions i.e, 5 different inits.
# Will save last model snapshot. 

def run_independent_solutions():

    EPOCHS = 10
    SAVE_PATH = 'IndependentSolutions/'

    for iter in tqdm_notebook(range(5)):
      # initialize weights
      keras.backend.clear_session()
      model = Model()
      #optimizer define with name
      opt = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-07, name='Adam'+str(iter))
      # compile
      model.compile(opt, 'sparse_categorical_crossentropy', metrics=['accuracy'])
      # train
      _ = model.fit(trainloader,
                  epochs=EPOCHS,
                  validation_data=testloader,
                  callbacks=[lr_callback],
                  verbose=0)
      # save final model snapshot
      model.save(SAVE_PATH+'smallcnn_independent_model_{}.h5'.format(iter))
      # evaluate
      loss, accuracy = model.evaluate(testloader, verbose=0)
      print("Test Error Rate: ", round((1-accuracy)*100, 2), '% | Iteration No: ', iter)

def run_independent_solutions(lr,ep,trainloader,testloader):

    EPOCHS = 10
    SAVE_PATH = 'IndependentSolutions/'

    for iter in tqdm_notebook(range(5)):
      # initialize weights
      keras.backend.clear_session()
      model = Model()
      #optimizer define with name
      opt = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=ep, name='Adam'+str(iter))
      # compile
      model.compile(opt, 'sparse_categorical_crossentropy', metrics=['accuracy'])
      # train
      _ = model.fit(trainloader,
                  epochs=EPOCHS,
                  validation_data=testloader,
                  callbacks=[lr_callback],
                  verbose=0)
      # save final model snapshot
      model.save(SAVE_PATH+'smallcnn_independent_model_{}.h5'.format(iter))
      # evaluate
      loss, accuracy = model.evaluate(testloader, verbose=0)
      print("Test Error Rate: ", round((1-accuracy)*100, 2), '% | Iteration No: ', iter)
#run_independent_solutions()

# Ensembles


# Deep Ensembles

In [12]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions(members):
	# make predictions
	yhats = [model.predict(x_test/255.) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = np.argmax(summed, axis=1)
	return result
 
# evaluate a specific number of members in an ensemble
def evaluate_n_members(members):
    accuracy_list = []
    for n_members in tqdm_notebook(range(1, len(members))):
        # select a subset of members
        subset = members[:n_members]
        print(len(subset))
        # make prediction
        yhat = ensemble_predictions(subset)
        # calculate accuracy
        accuracy_list.append(100*accuracy_score(y_test, yhat))

    return accuracy_list

def ensemble_predictions(members, test):
	# make predictions
	yhats = [model.predict(test/255.) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = np.argmax(summed, axis=1)
	return result

In [13]:
# Run the ensembles for plot
def run_ensembles_and_plot():
    members = [tf.keras.models.load_model(MODEL_PATH + model_ckpts[i]) 
                            for i in range(len(model_ckpts))]
    
    single_model = np.random.choice(members)
    accuracy_single_model = single_model.evaluate(testloader)[1] * 100
    
    accuracy_list = evaluate_n_members(members)
    
    accuracy_list.insert(0, accuracy_single_model)
    rng = [i for i in range(0, len(members))]
    
    plt.figure(figsize=(9,8))
    plt.plot(rng, accuracy_list, label='deep ensemble')
    plt.plot(rng, [accuracy_single_model]*len(rng), '--', label='single model')
    plt.title("Test accuracy as a function of ensemble size")
    plt.xlabel("Ensemble size")
    plt.ylabel("Test accuracy")
    plt.legend();
    plt.savefig('ensemble_func.png')
    
#run_ensembles_and_plot()

In [14]:
# Bayesian Optimization with k fold validation 
def iner_kfold(x_train, y_train, lr, ep):
    kf = KFold(n_splits = 3, random_state=3, shuffle=True)
    accuracy_list = []
    for train_index, test_index in kf.split(x_train):
        f_x_train, f_x_test = x_train[train_index], x_train[test_index]
        f_y_train, f_y_test = y_train[train_index], y_train[test_index]
        trainloader, testloader = get_loaders(f_x_train, f_y_train, f_x_test, f_y_test)
        run_independent_solutions(lr, ep,trainloader,testloader)
        model_ckpts = os.listdir(MODEL_PATH)
        members = [tf.keras.models.load_model(MODEL_PATH + model_ckpts[i]) 
                            for i in range(len(model_ckpts))]
        yhat = ensemble_predictions(members,f_x_test)
        accuracy = 100*accuracy_score(f_y_test, yhat)
        accuracy_list.append(accuracy)
    print('accuracy:', np.mean(accuracy_list), '+/-', np.std(accuracy_list))
    return np.mean(accuracy_list)

def kfold_opt(x_train, y_train):
    print("finding best hyper parameters")
    fit_with_partial = partial(iner_kfold, x_train, y_train)
    pbounds = { 'lr': (0.0001,0.01), 'ep': (1e-8,1e-5)}

    optimizer = BayesianOptimization(
        f=fit_with_partial,
        pbounds=pbounds,
        verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
        random_state=1,
    )

    optimizer.maximize(init_points=10, n_iter=10)


    for i, res in enumerate(optimizer.res):
        print("Iteration {}: \n\t{}".format(i, res))

    print(optimizer.max)
    return optimizer.max['params']['lr'], optimizer.max['params']['ep']




In [15]:
#claculating the result TPR and FPR
def tpr_fpr(y_true, y_prediction):

    cnf_matrix = confusion_matrix(y_true, y_prediction)
   
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float).sum()
    FN = FN.astype(float).sum()
    TP = TP.astype(float).sum()
    TN = TN.astype(float).sum()

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    return TPR, FPR

In [16]:
#writing the results to a file
def write_res(name, algo, cros_val, hp_lr, hp_ep, accu, tpr, fpr, preci, auc, train_time):
    file_name = name + '-' + algo + ".csv"
    print(file_name)
    f = open(file_name, "a")
    f.write(name+','+algo+','+str(cros_val)+','+str(hp_lr)+','+str(hp_ep)+','
            +str(accu)+','+str(tpr)+','+str(fpr)+','+str(preci)+','+str(auc)+','+str(train_time)+'\n')
    f.close()

In [17]:
#get AUC of multiclass results
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [18]:
# validate preformence result with kfold 
def kfold_val():
    f = open(n + "-paper_algo.csv", "w")
    f.write('name, algo, cros_val, hp_lr, hp_ep, accu, tpr, fpr, preci, auc, train_time\n')
    f.close()
    kf = KFold(n_splits=10, random_state=3, shuffle=True) #should be 10
    start1 = time.time()
    lr=0.001
    ep=1e-07
    accuracy_list = []
    training_times = []
    first_fold = True
    cros_iter = 1
    for train_index, test_index in kf.split(x):
        print("TRAIN:", train_index, "TEST:", test_index)
        f_x_train, f_x_test = x[train_index], x[test_index]
        f_y_train, f_y_test = y[train_index], y[test_index]
        start2 = time.time()
        if first_fold:
            #find best hyperparameters to use
            lr, ep = kfold_opt(f_x_train, f_y_train)
            first_fold = False
        
        trainloader, testloader = get_loaders(f_x_train, f_y_train, f_x_test, f_y_test) 
        run_independent_solutions(lr,ep,trainloader,testloader)
        end2 = time.time()
        model_ckpts = os.listdir(MODEL_PATH)
        members = [tf.keras.models.load_model(MODEL_PATH + model_ckpts[i]) 
                            for i in range(len(model_ckpts))]
        yhat = ensemble_predictions(members,f_x_test)
        ##### infer results and saving
        accuracy = 100*accuracy_score(f_y_test, yhat)
        TPR, FPR = tpr_fpr(f_y_test, yhat)
        precision = precision_score(f_y_test, yhat, average='weighted')
        auc = multiclass_roc_auc_score(f_y_test,yhat)
        accuracy_list.append(accuracy)
        train_time = end2 - start2
        training_times.append(train_time)
        print("accuracy: {}".format(accuracy))
        write_res(n, 'paper_algo', cros_iter, lr, ep, accuracy, TPR, FPR, precision, auc, train_time)
        cros_iter = cros_iter + 1
    end1 = time.time()
    print('accuracy:', np.mean(accuracy_list), '+/-', np.std(accuracy_list))
    print("avarage accuracy: {}".format(np.average(accuracy_list)) )   
    print("Network takes {:.3f} minuts to fold validate".format((end1 - start1)/60))

#kfold_val()

In [19]:
#model_list = ['cifar10', 'cifar100', 'mnist', 'flowers102', 'monkeys10', 'intel','cifar10_2', 'cifar100_2', 'mnist_2', 'flowers102_2', 'monkeys10_2', 'intel_2']

#model_list = ['cifar100', 'cifar100_2', 'flowers102', 'flowers102_2']
model_list = ['flowers102']
color = 0
for n in model_list:
    if n == 'cifar10':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar10()
    elif n == 'cifar10_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar10()
    elif n == 'cifar100':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar100()
    elif n == 'cifar100_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_cifar100()
    elif n == 'mnist':
        x,x2,y,y2,num_labels,img_shape,color = Ensembles_prepare_datasets.load_mnist()
    elif n == 'mnist_2':
        x1,x,y1,y,num_labels,img_shape,color = Ensembles_prepare_datasets.load_mnist()
    elif n == 'flowers102':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_flowers102()
    elif n == 'flowers102_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_flowers102()
        
    
    elif n == 'monkeys10':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_monkeys10()
    elif n == 'monkeys10_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_monkeys10()
    elif n == 'intel':
        x,x2,y,y2,num_labels,img_shape = Ensembles_prepare_datasets.load_intel()
    elif n == 'intel_2':
        x1,x,y1,y,num_labels,img_shape = Ensembles_prepare_datasets.load_intel()
    kfold_val()

TRAIN: [   1    2    3 ... 4091 4092 4093] TEST: [   0    6   21   22   33   34   38   59   69   70   72  111  115  121
  122  154  160  173  175  183  192  198  202  203  215  226  236  240
  250  261  272  273  285  287  318  322  323  325  328  349  370  371
  377  392  404  406  407  422  433  445  447  448  454  458  464  475
  499  509  528  529  532  537  549  557  563  575  583  591  602  603
  626  632  654  658  660  668  685  696  701  718  719  725  741  749
  768  779  781  785  800  805  818  820  821  849  871  877  880  901
  904  908  911  915  920  922  953  967  973  976  981  989 1008 1013
 1020 1031 1032 1041 1044 1049 1054 1061 1069 1089 1097 1099 1111 1113
 1119 1123 1130 1136 1150 1151 1162 1163 1165 1168 1182 1196 1198 1210
 1218 1260 1273 1281 1312 1336 1344 1354 1365 1371 1379 1390 1392 1400
 1407 1411 1414 1417 1440 1443 1444 1453 1460 1491 1502 1511 1547 1549
 1563 1610 1620 1627 1631 1634 1635 1651 1656 1660 1668 1671 1673 1675
 1706 1722 1726 1730 1731 17

  0%|          | 0/5 [00:00<?, ?it/s]

Test Error Rate:  85.34 % | Iteration No:  0
Test Error Rate:  83.47 % | Iteration No:  1
Test Error Rate:  87.54 % | Iteration No:  2
Test Error Rate:  82.41 % | Iteration No:  3
Test Error Rate:  84.04 % | Iteration No:  4


  0%|          | 0/5 [00:00<?, ?it/s]

Test Error Rate:  85.5 % | Iteration No:  0
Test Error Rate:  90.47 % | Iteration No:  1
Test Error Rate:  85.34 % | Iteration No:  2
Test Error Rate:  86.32 % | Iteration No:  3
Test Error Rate:  88.68 % | Iteration No:  4


...

accuracy: 22.73838630806846
flowers102-paper_algo.csv
TRAIN: [   0    1    2 ... 4090 4091 4093] TEST: [  12   13   17   32   35   44   52   77  117  125  133  141  176  182
  213  217  223  242  259  267  274  277  278  282  292  295  299  337
  344  363  367  368  372  375  383  418  426  434  435  438  455  470
  476  480  483  484  489  497  503  508  538  545  553  576  580  601
  607  621  622  625  641  647  655  657  659  672  684  690  704  728
  732  738  754  767  775  776  788  789  795  798  826  830  834  836
  848  858  860  864  865  887  900  939  964  968  970  974  983  994
 1003 1010 1019 1023 1024 1033 1045 1058 1062 1084 1086 1114 1143 1172
 1180 1184 1186 1191 1202 1229 1245 1252 1276 1291 1299 1316 1340 1356
 1359 1393 1396 1429 1438 1445 1447 1479 1486 1498 1505 1517 1519 1520
 1526 1528 1534 1541 1564 1575 1590 1596 1607 1614 1626 1628 1632 1633
 1642 1667 1684 1688 1694 1705 1709 1712 1717 1720 1738 1742 1744 1758
 1761 1762 1768 1782 1786 1789 1792 1803 1810

  0%|          | 0/5 [00:00<?, ?it/s]

Test Error Rate:  92.18 % | Iteration No:  0
Test Error Rate:  83.62 % | Iteration No:  1
Test Error Rate:  85.09 % | Iteration No:  2
Test Error Rate:  87.29 % | Iteration No:  3
Test Error Rate:  82.64 % | Iteration No:  4


C:\Users\User\.conda\envs\ML_proj\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 17.6039119804401
flowers102-paper_algo.csv
accuracy: 20.27288448923609 +/- 2.437223048201639
avarage accuracy: 20.27288448923609
Network takes 208.159 minuts to fold validate
